## Лабораторная работа ##

### Постановка задачи ###

Смоделируйте число страховых случаев, наступающих в портфеле из $N$ договоров страхования, эмпирическое распределение которых представлено в таблице, с помощью:
* распределения Пуассона;
* смешанного пуассоновского гамма-распределения;
* смешанного пуассоновского обратного гауссовского распределения;
* модели Лемера "хорошие - плохие риски" 

|Количество страховых случаев | Количество договоров |
:--------:|:-------:
| $k_1$ | $m_1$ |
| $k_2$ | $m_2$ |
| $k_3$ | $m_3$ |
|. . .  | . . . |
| $k_n$ | $m_n$ |

## Решение ##

### Реализованы следующие функци ###

In [1]:
import math
import pandas as pd
import scipy.stats as stats
import numpy as np

1. `get_poisson_params(df)` - принимает на вход исходную таблицу с данными (формат pandas
DataFrame) и возвращает параметр $\overline\lambda$ пуассоновского распределения.

$\lambda_{\text{e}} = \frac{1}{n} \sum_{i=1}^{n} (k_i \cdot m_i)$

In [2]:
def get_poisson_params(df):
    return sum(df['k'] * df['m']) / sum(df['m'])

2. `get_poisson_gamma_params(df)` - принимает на вход исходную таблицу с данными (формат pandas
DataFrame) и возвращает параметры $\overline a$ и $\overline  b$ смешанного пуассоновского гамма-распределения.<br>

$k_{\text{e}} = \frac{1}{n} \sum_{i=1}^{n} (k_i \cdot m_i)$

$k_{\text{square(e)}} = \frac{1}{n} \sum_{i=1}^{n} (k_i^2 \cdot m_i)$

$S_{\text{square(k)}} = k_{\text{square(e)}} - k_{\text{e}}^2$

$a_{\text{e}} = \frac{k_{\text{e}}^2}{S_{\text{square(k)}} - k_{\text{e}}}$

$b_{\text{e}} = \frac{k_{\text{e}}}{S_{\text{square(k)}} - k_{\text{e}}}$

In [3]:
def get_poisson_gamma_params(df):
    n = sum(df['m'])

    k_e = sum(df['k'] * df['m']) / n
    k_square_e = sum(df['k'] ** 2 * df['m']) / n
    S_square_k = k_square_e - k_e**2
    

    a_e = k_e**2/(S_square_k - k_e)
    b_e = k_e/(S_square_k - k_e)
    
    return a_e, b_e

3. `get_poisson_gaussian_params(df)` - принимает на вход исходную таблицу с данными (формат pandas
DataFrame) и возвращает параметры $\overline g$ и $\overline  h$ смешанного пуассоновского обратного гауссовского рас
пределения.

$n = \sum_{i=1}^{n} m_i$

$k_{\text{e}} = \frac{1}{n} \sum_{i=1}^{n} (k_i \cdot m_i)$

$k_{\text{square(e)}} = \frac{1}{n} \sum_{i=1}^{n} (k_i^2 \cdot m_i)$

$S_{\text{square(k)}} = k_{\text{square(e)}} - k_{\text{e}}^2$

$g_{\text{e}} = k_{\text{e}}$

$h_{\text{e}} = \left( \frac{S_{\text{square(k)}}}{k_{\text{e}}} \right) - 1$


In [4]:
def get_poisson_gaussian_params(df):
    n = sum(df['m'])


    k_e = sum(df['k'] * df['m']) / n
    k_square_e = sum(df['k'] ** 2 * df['m']) / n
    S_square_k = k_square_e - k_e**2


    g_e = k_e
    h_e = (S_square_k / k_e) - 1 
    
    return g_e, h_e

4. `get_lemaire_params(df)` - принимает на вход исходную таблицу с данными (формат pandas
DataFrame и возвращает параметры $\overline{a_1}$, $\overline{\lambda_1}$ и $\overline{\lambda_2}$ модели Лемера.

$\lambda_{1,2(\text{e})} = \frac{A \pm \sqrt{A^2 - 4 \cdot B}}{2}$

$a_{1 (\text{e})} = \frac{a - \lambda_{2(\text{e})}}{\lambda_{1 (\text{e})} - \lambda_{2 (\text{e})}}$

In [5]:
def get_lemaire_params(df):
    n = sum(df['m'])
    
    k_e = sum(df['k'] * df['m']) / n
    k_square_e = sum(df['k'] ** 2 * df['m']) / n
    k_cubic_e = sum(df['k'] ** 3 * df['m']) / n

    a = k_e
    b = k_square_e - k_e
    c = k_cubic_e - 3 * k_square_e + 2 * k_e
    A = (c - a * b) / (b - a ** 2)
    B = (a * c - b ** 2) / (b - a ** 2)

  
    λ1_e = (A - np.sqrt(A**2 - 4 * B)) / 2
    λ2_e = (A + np.sqrt(A**2 - 4 * B)) / 2
    a1_e = (a - λ2_e) / (λ1_e - λ2_e)
    
    return a1_e, λ1_e, λ2_e

5. Функция, которые для каждого из распределений принимают на вход исходные данные, а также найденные параметры и возвращают таблицу (формат pandas DataFrame) с исходными данными и теоретическими вероятностями $p^t_i$ частотами $m^t_i$(итоговая, т.е. после схлопывания строк с объемом
менее 5):
* `get_poisson_data`(df, $\lambda$)

$p_k = \frac{e^{-\lambda_{(\text{e})}} \cdot \lambda_{(\text{e})}^k}{k!}$

$m_{\text{e}} = n \cdot p_k$

* `get_poisson_gamma_data`(df, $\overline a$, $\overline b$)

$p_k = C_{k+a+1}^k \cdot \left( p^a \cdot (1 - p)^k \right)$, где $p = \frac{1}{1 + b}$

$m_{\text{e}} = n \cdot p_k$

* `get_poisson_gaussian_data`(df, $\overline g$, $\overline h$)

Для $k = 0$:

   $p_0 = e^{\frac{g_{\text{e}} \cdot (1 - \sqrt{1 + 2h_{\text{e}}})}{h_{\text{e}}}}$

Для $k = 1$:

   $p_1 = p_0 \cdot \frac{g_{\text{e}}}{\sqrt{1 + 2h_{\text{e}}}}$

Для $k > 1$:

   $p_k = \frac{p_{k-1} \cdot h_{\text{e}} \cdot (k-1) \cdot (2k-3) + p_{k-2} \cdot g_{\text{e}}^2}{(1 + 2h_{\text{e}}) \cdot k \cdot (k-1)}$
   

$m_{\text{e}} = n \cdot p_k$

* `get_lemaire_data`(df, $\overline{a_1}$, $\overline{\lambda_1}$, $\overline{\lambda_2}$)

$a_{2 (\text{e})} = 1 - a_{1 (\text{e})}$

$p_k = a_{1 (\text{e})} \cdot \frac{e^{-\lambda_{1 (\text{e})}} \cdot \lambda_{1 (\text{e})}^k}{k!} + a_{2 (\text{e})} \cdot \frac{e^{-\lambda_{2 (\text{e})}} \cdot \lambda_{2 (\text{e})}^k}{k!}$

$m_{\text{e}} = n \cdot p_k$

In [6]:
def xlop(df):
    if (len(df) < 4):
        return df
    result = df.copy()
    
    while len(result) > 1 and result.iloc[-1]['m'] < 5:
        last_but_one_idx = result.index[-2]
        for column in result.columns:
            result.loc[last_but_one_idx, column] += result.iloc[-1][column]
        result = result.drop(result.index[-1])
    
    return result

# def xlop(df):
#     if len(df) < 4:
#         return df
    
#     # Создаем копию DataFrame
#     result = df.copy()
    
#     # Объединяем строки, где 'm' < 5, с предыдущей строкой
#     while (result['m'] < 5).any():
#         # Находим индексы строк, где 'm' < 5
#         idx_to_merge = result[result['m'] < 5].index
#         if len(idx_to_merge) == 0:
#             break
        
#         # Объединяем последнюю строку с предыдущей
#         last_idx = idx_to_merge[-1]
#         prev_idx = last_idx - 1
        
#         # Суммируем значения в строках
#         result.loc[prev_idx, 'm'] += result.loc[last_idx, 'm']
#         result.loc[prev_idx, 'k'] = result.loc[prev_idx, 'k']  # Можно оставить предыдущее значение 'k'
        
#         # Удаляем последнюю строку
#         result = result.drop(last_idx)
    
#     return result

# def get_poisson_data(df, λ_e):
#     poisson_data = df.copy()
#     n = sum(df['m'])
    
#     poisson_data['p'] = stats.poisson.pmf(df['k'], λ_e)
#     poisson_data['m_e'] = round(poisson_data['p'] * n)

#     return xlop(poisson_data)


# def get_poisson_gamma_data(df, a_e, b_e):
#     poisson_gamma_data = df.copy()
#     n = sum(df['m'])
    
#     poisson_gamma_data['p'] = stats.nbinom.pmf(df['k'], 
#                                                a_e, b_e / (1 + b_e))

#     poisson_gamma_data['m_e'] = round(poisson_gamma_data['p'] * n)
    
#     return poisson_gamma_data



# def get_poisson_gaussian_data(df, g_e, h_e):
#     poisson_gaussian_data = df.copy()
#     n = sum(df['m'])
    

#     p = np.zeros(df['k'].max() + 1)
#     for k in range(0, p.size):
#         if (k == 0):
#             p[0] = np.exp(g_e * (1 - np.sqrt(1 + 2 * h_e)) / h_e)
#         elif (k == 1):
#             p[1] = p[0] * g_e / np.sqrt(1 + 2 * h_e)
#         else:
#             p[k] = ((p[k - 1] * h_e * (k - 1) * (2 * k - 3) + 
#                     p[k - 2] * g_e * g_e)) / \
#                         ((1 + 2 * h_e) * k * (k - 1))
    
#     poisson_gaussian_data['p'] = p[df['k']]
#     poisson_gaussian_data['m_e'] = round(poisson_gaussian_data['p'] * n)
    
#     return xlop(poisson_gaussian_data)


# def get_lemaire_data(df, a1_e, λ1_e, λ2_e):
#     lemaire_data = df.copy()
#     n = sum(df['m'])

#     a2_e = 1 - a1_e
    
 
#     lemaire_data['p'] = a1_e * stats.poisson.pmf(df['k'], λ1_e) + \
#                     (a2_e) * stats.poisson.pmf(df['k'], λ2_e)
#     lemaire_data['m_e'] = round(lemaire_data['p'] * n)
    
#     return xlop(lemaire_data)

In [19]:
def calculate_m_e_and_xlop(df, p_column):
    n = sum(df['m'])
    df['m_e'] = round(df[p_column] * n)
    return xlop(df)

def get_poisson_data(df, λ_e):
    """
    Вычисляет теоретические вероятности и частоты для распределения Пуассона.
    
    Параметры:
    - df: DataFrame с исходными данными.
    - λ_e: Параметр распределения Пуассона.
    
    Возвращает:
    - DataFrame с теоретическими вероятностями и частотами.
    """
    if (df['k'] < 0).any() or (df['m'] < 0).any():
        raise ValueError("Значения 'k' и 'm' должны быть неотрицательными.")
    
    poisson_data = df.copy()
    poisson_data['p'] = stats.poisson.pmf(df['k'], λ_e)
    return calculate_m_e_and_xlop(poisson_data, 'p')

def get_poisson_gamma_data(df, a_e, b_e):
    poisson_gamma_data = df.copy()
    poisson_gamma_data['p'] = stats.nbinom.pmf(df['k'], a_e, b_e / (1 + b_e))
    return calculate_m_e_and_xlop(poisson_gamma_data, 'p')

def get_poisson_gaussian_data(df, g_e, h_e):
    poisson_gaussian_data = df.copy()
    max_k = df['k'].max()
    p = np.zeros(max_k + 1)
    
    # Вычисление вероятностей
    p[0] = np.exp(g_e * (1 - np.sqrt(1 + 2 * h_e)) / h_e)
    if max_k >= 1:
        p[1] = p[0] * g_e / np.sqrt(1 + 2 * h_e)
    for k in range(2, max_k + 1):
        p[k] = ((p[k - 1] * h_e * (k - 1) * (2 * k - 3) + 
                 p[k - 2] * g_e * g_e)) / \
                ((1 + 2 * h_e) * k * (k - 1))
    
    poisson_gaussian_data['p'] = p[df['k']]
    return calculate_m_e_and_xlop(poisson_gaussian_data, 'p')

def get_lemaire_data(df, a1_e, λ1_e, λ2_e):
    lemaire_data = df.copy()
    a2_e = 1 - a1_e
    lemaire_data['p'] = a1_e * stats.poisson.pmf(df['k'], λ1_e) + \
                        a2_e * stats.poisson.pmf(df['k'], λ2_e)
    return calculate_m_e_and_xlop(lemaire_data, 'p')

6. Функция `get_pvalue(df_new)`, которая принимает на вход новый DataFrame и возвращает значение $p_{value}$.

$\chi^2 = \sum_{i} \frac{(m_i - m_{\text{e}})^2}{m_{\text{e}}}$

$p_{value} = 1 - \text{CDF}_{\chi^2}(\chi^2, \text{df})$


In [8]:
def get_pvalue(df_new, params_count):
    df_filtered = df_new[df_new['m_e'] > 0]

    χ_square = sum((df_filtered['m'] - df_filtered['m_e'])**2 / df_filtered['m_e'])
    m = params_count
    df = len(df_filtered) - m - 1
    p_value = 1 - stats.chi2.cdf(x = χ_square, df = df)
    
    return p_value

6. Функция `get_result(α, p_value_poisson, p_value_poisson_gamma, p_value_poisson_gaussian,
p_value_lemaire)`, которая принимает на вход уровень значимости и возвращает таблицу с тремя полями:

| Название метода       |$p_{value}$| Рейтинг модели |
|-----------------------|---------|----------------|
|       Пуассон         |         |                |
|       Пуассон-Гамма   |         |                |
|       Пуассон-Гаусс   |         |                |
|       Лемер           |         |                |

- Название метода
- $p_{value}$
- Рейтинг модели: 1 - самая лучшая. Если модель не подходит по критерию Пирсона, то вместо рейтинга - "Не подходит".

In [9]:
def get_result(α, p_value_poisson, p_value_poisson_gamma, p_value_poisson_gaussian, p_value_lemaire):
    results = pd.DataFrame({
        'Метод': ['Пуассон', 'Пуассон-Гамма', 'Пуассон-Гаусс', 'Лемер'],
        'p_value': [p_value_poisson, 
                    p_value_poisson_gamma, 
                    p_value_poisson_gaussian, 
                    p_value_lemaire]
    })
    
    results['Рейтинг'] = 'Не подходит'
    suitable_models = results[results['p_value'] > α].sort_values('p_value', ascending=False)
    
    if len(suitable_models) > 0:
        for i, idx in enumerate(suitable_models.index):
            results.loc[idx, 'Рейтинг'] = i + 1
    def format_p_value(p_value):
        return f'{p_value:.12f}'

    results['p_value'] = results['p_value'].apply(format_p_value)

    return results

### Проверка на данных из лекции (до схлопывания строк) ###

#### Распределение Пуассона ####

In [10]:
df = pd.DataFrame({
    'k': [0, 1, 2],
    'm': [29196, 184, 2]
})
print(df.to_string(index=False), end="\n\n")

λ_e = get_poisson_params(df)
print("λ_e =", λ_e, end="\n\n")

poisson_data = get_poisson_data(df, λ_e)
print(poisson_data.to_string(index=False), end="\n\n")

p_value = get_pvalue(poisson_data, 1)
print("p_value =", p_value, end="\n\n")


 k     m
 0 29196
 1   184
 2     2

λ_e = 0.006398475256960043

 k     m        p     m_e
 0 29196 0.993622 29195.0
 1   184 0.006358   187.0
 2     2 0.000020     1.0

p_value = 0.3059306375947972



#### Смешанное пуассоновское-гамма распределение ####

In [11]:
df = pd.DataFrame({
    'k': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'm': [1624, 490, 208, 98, 48, 23, 10, 5, 3, 2, 1]
})
print(df.to_string(index=False), end="\n\n")

a_e, b_e = get_poisson_gamma_params(df)
print("a =", a_e)
print("b =", b_e, end="\n\n")

poisson_gamma_data = get_poisson_gamma_data(df, a_e, b_e)
print(poisson_gamma_data.to_string(index=False), end="\n\n")

p_value = get_pvalue(poisson_gamma_data, 2)
print("p_value =", p_value, end="\n\n")


 k    m
 0 1624
 1  490
 2  208
 3   98
 4   48
 5   23
 6   10
 7    5
 8    3
 9    2
10    1

a = 0.5845911753468382
b = 0.8878434295473141

 k    m        p    m_e
 0 1624 0.643385 1616.0
 1  490 0.199231  500.0
 2  208 0.083614  210.0
 3   98 0.038158   96.0
 4   48 0.018113   46.0
 5   23 0.008798   22.0
 6   10 0.004337   11.0
 7    5 0.002161    5.0
27    6 0.001912    5.0

p_value = 0.9939666432602774



#### Смешанное пуассоновское - обратное гауссовское распределение ####

In [12]:
df = pd.DataFrame({
        'k': [0, 1, 2, 3, 4],
        'm': [205925, 9940, 1054, 114, 13]
})
print(df.to_string(index=False), end="\n\n")

g_e, h_e = get_poisson_gaussian_params(df)
print("g =", g_e)
print("h =", h_e, end="\n\n")

poisson_gaussian_data = get_poisson_gaussian_data(df, g_e, h_e)
print(poisson_gaussian_data.to_string(index=False), end="\n\n")

p_value = get_pvalue(poisson_gaussian_data, 2)
print("p_value =", p_value, end="\n\n")


 k      m
 0 205925
 1   9940
 2   1054
 3    114
 4     13

g = 0.05732425384480709
h = 0.17961514496567355

 k      m        p      m_e
 0 205925 0.948442 205856.0
 1   9940 0.046634  10122.0
 2   1054 0.004228    918.0
 3    114 0.000577    125.0
 4     13 0.000096     21.0

p_value = 1.0896157963680153e-06



#### Смешанное Лемера "хорошие - плохие риски"  ####

In [16]:
df = pd.DataFrame({
        'k': [0, 1, 2, 3, 4],
        'm': [205925, 9940, 1054, 114, 13]
})
print(df.to_string(index=False), end="\n\n")

a1_e, λ1_e, λ2_e = get_lemaire_params(df)
print("a1 =", a1_e)
print("λ1 =", λ1_e)
print("λ2 =", λ2_e, end="\n\n")

lemaire_data = get_lemaire_data(df, a1_e, λ1_e, λ2_e)
print(lemaire_data.to_string(index=False), end="\n\n")

p_value = get_pvalue(lemaire_data, 3)
print("p_value =", p_value, end="\n\n")



 k      m
 0 205925
 1   9940
 2   1054
 3    114
 4     13

a1 = 0.8914720976725681
λ1 = 0.021919831822791308
λ2 = 0.3481439756858097

 k      m        p      m_e
 0 205925 0.948764 205925.0
 1   9940 0.045792   9939.0
 2   1054 0.004853   1053.0
 3    114 0.000540    117.0
 4     13 0.000047     10.0

p_value = 0.32269966404264216



## Пример работы программы ##

### Проверка на данных их книги (после схлопывания строк) ###

In [14]:
df = pd.DataFrame({
        'k': [0, 1, 2, 3, 4],
        'm': [205925, 9940, 1054, 114, 13]
})

α = 0.05

λ = get_poisson_params(df)
a, b = get_poisson_gamma_params(df)
g, h = get_poisson_gaussian_params(df)
a1, λ1, λ2 = get_lemaire_params(df)

result = get_result(
    α,
    get_pvalue(
        get_poisson_data(df, λ), 1),
    get_pvalue(
        get_poisson_gamma_data(df, a, b), 2),
    get_pvalue(
        get_poisson_gaussian_data(df, g, h), 2),
    get_pvalue(
        get_lemaire_data(df, a1, λ1, λ2), 3)
)
result

,Метод,p_value,Рейтинг
0,Пуассон,0.000000000000,Не подходит
1,Пуассон-Гамма,0.198921613852,2
2,Пуассон-Гаусс,0.000001089616,Не подходит
3,Лемер,0.322699664043,1
